<a href="https://colab.research.google.com/github/raminnourizade/Proposal/blob/main/egg_ds004504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 25.3 MB/s eta 0:00:00


In [2]:
 pip install -q kaggle

In [12]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"raminnourizadeh","key":"b0cd465e908eb8b5f9234b3ae3a80e68"}'}

In [4]:
! mkdir ~/.kaggle

In [13]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [14]:
! kaggle datasets list

ref                                                 title                                    size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  --------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                  Vehicle Sales Data                       19MB  2024-02-21 20:16:17          11993        226  1.0              
sudarshan24byte/online-food-dataset                 Online Food Dataset                       3KB  2024-03-02 18:50:30           2015         38  0.9411765        
arnavvvvv/spotify-music                             Top Spotify Songs                        47KB  2024-03-06 05:20:29           2855         50  1.0              
tarunrm09/climate-change-indicators                 Climate change Indicators                34KB  2024-02-22 08:53:54           6309        134  1.0              
nbroad/gemma-rew

In [15]:
!kaggle datasets download -d bahoho/alzheimer-derivative-ds004504

100% 2.36G/2.37G [00:36<00:00, 29.2MB/s]
100% 2.37G/2.37G [00:36<00:00, 70.1MB/s]


In [16]:
! mkdir train

In [17]:
! unzip '/content/alzheimer-derivative-ds004504.zip' -d train



Archive:  /content/alzheimer-derivative-ds004504.zip
  inflating: train/derivatives/sub-001/eeg/sub-001_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-002/eeg/sub-002_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-003/eeg/sub-003_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-004/eeg/sub-004_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-005/eeg/sub-005_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-006/eeg/sub-006_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-007/eeg/sub-007_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-008/eeg/sub-008_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-009/eeg/sub-009_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-010/eeg/sub-010_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-011/eeg/sub-011_task-eyesclosed_eeg.set  
  inflating: train/derivatives/sub-012/eeg/sub-012_task-eyesclosed_eeg.set  
  inflating: train/deri

In [33]:
import os
import mne
import glob


In [34]:
def get_set_files(root_dir):
    pattern = os.path.join(root_dir, '**', '*.set')
    set_files = glob.glob(pattern, recursive=True)
    return set_files


In [36]:
directory_path = '/content/train/derivatives'
set_files = get_set_files(directory_path)
# files_and_directories =  os.listdir(directory_path)
# directories=[d for d in files_and_directories if os.path.isdir(os.path.join(directory_path, d)) and d.startswith('sub')]


In [37]:
set_files

['/content/train/derivatives/sub-051/eeg/sub-051_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-004/eeg/sub-004_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-008/eeg/sub-008_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-043/eeg/sub-043_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-006/eeg/sub-006_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-087/eeg/sub-087_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-045/eeg/sub-045_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-070/eeg/sub-070_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-007/eeg/sub-007_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-030/eeg/sub-030_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-014/eeg/sub-014_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-083/eeg/sub-083_task-eyesclosed_eeg.set',
 '/content/train/derivatives/sub-072/eeg/sub-072_task-eyesclosed_eeg.set',
 '/content/train/derivati

In [26]:
raw = mne.io.read_raw_eeglab('/content/train/derivatives/sub-002/eeg/sub-002_task-eyesclosed_eeg.set',preload=True)

In [29]:
events, event_id = mne.events_from_annotations(raw)


In [32]:
event_id

{}

In [28]:
raw

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,22 points
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


In [47]:
all_epochs = []

for (i,sfile) in enumerate( set_files):
  raw = mne.io.read_raw_eeglab(sfile,preload=True)
  raw.set_eeg_reference(ref_channels=[])



EEG channel type selected for re-referencing
EEG data marked as already having the desired reference.


<ipython-input-47-b0a824995817>:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(sfile,preload=True)


EEG channel type selected for re-referencing
EEG data marked as already having the desired reference.


<ipython-input-47-b0a824995817>:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(sfile,preload=True)


EEG channel type selected for re-referencing
EEG data marked as already having the desired reference.
EEG channel type selected for re-referencing
EEG data marked as already having the desired reference.
EEG channel type selected for re-referencing
EEG data marked as already having the desired reference.
EEG channel type selected for re-referencing


...

<ipython-input-47-b0a824995817>:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(sfile,preload=True)
<ipython-input-47-b0a824995817>:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(sfile,preload=True)


EEG channel type selected for re-referencing
EEG data marked as already having the desired reference.
EEG channel type selected for re-referencing


<ipython-input-47-b0a824995817>:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(sfile,preload=True)
<ipython-input-47-b0a824995817>:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(sfile,preload=True)


EEG data marked as already having the desired reference.
EEG channel type selected for re-referencing
EEG data marked as already having the desired reference.


<ipython-input-47-b0a824995817>:4: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(sfile,preload=True)
